## Create FHIR R4 CapStatement Resource


### Outline:

- Source excel with requirements
- pandas to convert in python Ordered Dict
- build json
- generate narrative using Jinja2 templates

### Prerequisites:

- Python 3.6 or greater

## STEP1: Choose Source Spreadsheet to use

*note:  Jupyteralab and widgets issues: see https://stackoverflow.com/questions/4
9542417/how-to-get-ipywidgets-working-in-jupyter-lab for solution 

In [ ]:
#******************** NEED TO UPDATE WHEN Adding new IGS ************************************************

from os import name as os_name
my_base = '/Users/ehaas/' if os_name == 'posix' else '//ERICS-AIR-2/ehaas/'

in_path_list = [
        "---pick one below---",
        "Documents/Python/MyNotebooks/CapStatement/temp_source_spreadsheets/test-spreadsheet.xlsx",
        "Documents/FHIR/US-Core-R4/input/resources_spreadsheets/uscore-client.xlsx",
        "Documents/FHIR/US-Core-R4/input/resources_spreadsheets/uscore-server.xlsx",
        'Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/alert-initiator.xlsx',
         'Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/notification-forwarder.xlsx',
         'Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/alert-receiver.xlsx',
         'Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/query-responder.xlsx',
         'Documents/FHIR/Davinci-Alerts/input/resources/source-data/capstatements-spreadsheets/query-requester.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Consumer_Client.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Reporter_Client.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Consumer_Server.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Producer_Client.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Producer_Server.xlsx',
         'Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_Receiver_Server.xlsx',
         "Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_GIC_Reporter_Client.xlsx",
          "Documents/FHIR/Davinci-DEQM/input/resources/source-data/DEQM_Capability_Statement_GIC_Receiver_Server.xlsx",
         'C:/Users/Administrator/Downloads/plan-net-server.xlsx',
        'Documents/FHIR/Davinci-CDEX/input/resources-spreadsheet/data-source-client.xlsx',
        'Documents/FHIR/Davinci-CDEX/input/resources-spreadsheet/data-source-server.xlsx',
        'Documents/FHIR/Davinci-CDEX/input/resources-spreadsheet/data-consumer-client.xlsx',
        'Documents/FHIR/Davinci-CDEX/input/resources-spreadsheet/data-consumer-server.xlsx',
        'Documents/FHIR/Argo-PL/input/resources-spreadsheet/client.xlsx',
        'Documents/FHIR/Argo-PL/input/resources-spreadsheet/server.xlsx',   
        ]


# ----------spreadsheet source---------------
from IPython.display import display as Display, HTML, Markdown, Javascript
from ipywidgets import Dropdown
menu = Dropdown(
       options=[my_base + x for x in in_path_list],
       description='Choose Spreadsheet Source file',
       style = {'description_width': 'initial',},
       layout={'width': 'initial'},
        )


menu

## STEP2:  *CLICK HERE* and then 'Select Run Selected Cell and All Below'  from menu bar to continue"

In [ ]:
xls = menu.value
xls

### Import FHIRClient and other libraries

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
from fhir.resources import construct_fhir_element
from json import dumps, loads
from requests import get, post, put
from pathlib import Path
from IPython.display import display as Display, HTML, Markdown, Javascript
import ipywidgets as widgets
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime, date, timezone, timedelta
from jinja2 import Environment, FileSystemLoader, select_autoescape
from stringcase import snakecase, titlecase
from commonmark import commonmark
from htmlmin import minify

####  Assign Global Variables

Here is where we assign all the global variables for this example such as the canonical base and project information

In [ ]:
fhir_base_url = 'http://hl7.org/fhir/'
f_jurisdiction =  construct_fhir_element('CodeableConcept',{
      "coding" : [
        {
          "system" : "urn:iso:std:iso:3166",
          "code" : "US"
        }
      ]
    })

conf_url = 'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation'
combo_url = 'http://hl7.org/fhir/StructureDefinition/capabilitystatement-search-parameter-combination'
sp_specials = {'us-core-includeprovenance':'http://hl7.org/fhir/us/core/SearchParameter/us-core-includeprovenance'}  # dict to for SP to get right canonicals, may use spreadsheet or package file in future.

none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False', None,]

sep_list = (',', ';', ' ', ', ', '; ')


timezone_offset = -8.0  # Pacific Standard Time (UTC−08:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
f_now = datetime.now(tzinfo)
f_now

#### To PascalCase Utility

In [ ]:
def kebab_to_pascal(word):
    return ''.join(x.capitalize() for x in word.split('-'))

#### Set Simple Attributes

In [ ]:
def set_attr(v=None):
    if v and v not in none_list:
        return v
    else:
        return

#### Set List Attributes

In [ ]:
def set_attr_list(v=None):
    if v:
        return [i for i in v.split(",")]
    else:
        return

#### Conformance Extension

In [ ]:
def get_conf(conf='MAY',as_dict=False):
    if as_dict:
        return [construct_fhir_element('Extension',dict(
            url = conf_url,
            valueCode = conf
            )).json()]
    else:
        return [construct_fhir_element('Extension',dict(
            url = conf_url,
            valueCode = conf if conf else "MAY"
            ))]
        

#### Primitive Conformance Extension

In [ ]:
def get_prim_ext(conf=None):
    if not set(conf).issubset(none_list):
        conf_list = []
        try: # is comma sep string
            conf = conf.split(',')
        except AttributeError: # is list
            pass         
        for i in conf:
            if i not in none_list:
                conf_ext = construct_fhir_element('FHIRPrimitiveExtension', dict(
                    extension = get_conf(conf=i),
                    ))
            else:
                conf_ext = None
            conf_list.append(conf_ext)
        
        return conf_list
    else:
        return

### Addin Extensions

In [ ]:
def get_addin_ext(py_ext, json_ext):            
    print(py_ext)
    if json_ext:   # ie not ''
        addin_ext = construct_fhir_element('Extension',loads(json_ext))            
        # addin_ext.extension =  get_conf('SHALL') violates invariant   - DONT USE           
        print(addin_ext)                 
        py_ext.append(addin_ext) # add in other extensions
    print(py_ext)
    return py_ext

### validate

In [ ]:
# *********************** validate Resource ********************************

def validate(r):

    #fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server = 'http://hapi.fhir.org/baseR4'
    fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1'
    
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
      # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
        )
    
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{fhir_test_server}/Questionnaire/$validate', params = params, headers = headers, data = r.json())
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Get Cap Statement input data

function to convert to dataframe series to namedtuple for easy peasy dot notation use.

#### first the config data

In [ ]:
df = read_excel(xls,'config',na_filter = False,index_col=0)  # use the index_col = 0 for setting the first rwo as the index

df

#### assign globals e.g. publisher parameter etc...

In [ ]:
#df.[Column].[row] to get a value df.loc[[row],[Column]] or df.at[[column],[col]] works too
df.Value.source #, df.loc['source','Value'], df.at['source' ,'Value']

In [ ]:

ig_source_path = df.Value.source
ig_package_tar_path =  df.Value.packagepath
# --------- ig specific variable -------------------
pre = df.Value.pre  # for Titles - not sure this is actually used
canon = df.Value.canon # don't forget the slash  - fix using os.join or path
#
publisher = df.Value.publisher
#
publisher_endpoint = dict(
                    system = df.Value.publishersystem,
                    value = df.Value.publishervalue,
                  )

pprint(publisher_endpoint)


#### Get IG Names

until able to support primitive extensions in pyfhir

In [ ]:
def get_igs():
    ig_dict = {}
    df_igs = read_excel(xls,'igs',na_filter = False)
    for ig in df_igs.itertuples(index=True):
        try:
            ig_dict[ig.canonical] = (ig.name, ig.canonical, ig.conformance)
        except Exception as e:
            print(e)
    return ig_dict

ig_dict = get_igs()
shall_igs = {k:v for k,v in ig_dict.items() if v[2] == "SHALL"}
should_igs = {k:v for k,v in ig_dict.items() if v[2] == "SHOULD"}

ig_dict, shall_igs, should_igs

#### then the meta sheet

In [ ]:
df = read_excel(xls,'meta',na_filter = False)

df

#### Create NamedTuple from df to use dot notation

In [ ]:
d = dict(zip(df.Element, df.Value))
meta = namedtuple("Meta", d.keys())(*d.values())      
         
meta.id


### Create CS instance

In [ ]:
cs = construct_fhir_element('CapabilityStatement',dict(
id = meta.id,
url = f'{canon}CapabilityStatement/{meta.id}',
version = meta.version,
name = f'{kebab_to_pascal(meta.id)}CapabilityStatement',
title = f'{titlecase(meta.id).replace("Us ", "US ")} CapabilityStatement',
status = 'active',
experimental = False,
date = f_now, # as FHIRDate
publisher = publisher,
contact = [construct_fhir_element('ContactDetail', {"telecom" : [ publisher_endpoint ] })],
description = meta.description,
jurisdiction = [f_jurisdiction],
kind = 'requirements',
fhirVersion = meta.fhirVersion,
format = set_attr_list(meta.format),
format__ext = get_prim_ext(meta.format_conf),
patchFormat = set_attr_list(meta.patchFormat),
patchFormat__ext = get_prim_ext(meta.patchFormat_conf),
))

print(cs.json(indent=2))

#### Add in Rest

In [472]:
# get primitive extension

cs.patchFormat__ext[0].extension[0].valueCode

'SHOULD'

In [ ]:
def get_sys_op():
    op_list = []
    df_op = read_excel(xls,'ops',na_filter = False)
    for i in df_op.itertuples(index=True):
        if i.type == 'system':
            op = CS.CapabilityStatementRestResourceOperation()
            op.name = i.name 
            op.definition = i.definition
            op.extension = get_conf(i.conf)           
            op_list.append(op.json())
    return op_list if op_list else None


def get_rest_ints():
    ri_list = []
    df_ri = read_excel(xls,'rest_interactions',na_filter = False)
    for i in df_ri.itertuples(index=True):
        ri = construct_fhir_element('CapabilityStatementRestInteraction', dict(
        code = i.code,
        documentation = i.doc if i.doc not in none_list else None,
        extension = get_conf(i.conf),
        ))                         
        print(ri.json(indent=2))
        ri_list.append(ri.json())        
    return ri_list  if ri_list else None

rest = construct_fhir_element('CapabilityStatementRest',(dict(
    mode = meta.mode,
    documentation = meta.documentation,
    security = dict(
        description = meta.security
        ) if meta.security else None,
    interaction = get_rest_ints(),
    operation = get_sys_op()
    )))
                              
cs.rest = [rest]

print(cs.json(indent=2))

#### Add in IG, and Capstatements and their conformance

In [ ]:
df_igs = read_excel(xls,'igs',na_filter = False)
cs.implementationGuide = set_attr([ig.canonical for ig in df_igs.itertuples(index=True)])
#print([ig.conformance for ig in df_igs.itertuples(index=True) if ig.imports not in none_list])
cs.implementationGuide__ext = get_prim_ext([ig.conformance for ig in df_igs.itertuples(index=True)])

df_capstatements = read_excel(xls,'capstatements',na_filter = False)
cs.imports = set_attr([ig.canonical for ig in df_capstatements.itertuples(index=True) if ig.imports not in none_list])
cs.imports__ext = get_prim_ext([ig.conformance for ig in df_capstatements.itertuples(index=True)])
cs.instantiates = set_attr([ig.canonical for ig in df_capstatements.itertuples(index=True) if ig.instantiates not in none_list])
cs.instantiates__ext = get_prim_ext([ig.conformance for ig in df_capstatements.itertuples(index=True)])

print(cs.json(indent=2))

#### add Resources

- read sheets for resource attributes, interaction attributes,  search attributes, profiles, and combo search parameters

In [ ]:
df_resources = read_excel(xls,'resources',na_filter = False)
df_resources = df_resources[df_resources.type.str[0] != '!']
df_resources

In [ ]:
df_profiles = read_excel(xls,'profiles',na_filter = False)  #df1 = df[df.Hostname.str[0] != "abc"]
df_profiles = df_profiles[df_profiles.Profile.str[0] != '!']
df_profiles

In [ ]:
df_i = read_excel(xls,'interactions',na_filter = False)
df_sp = read_excel(xls,'sps',na_filter = False)
df_combos = read_excel(xls,'sp_combos',na_filter = False)
df_op = read_excel(xls,'ops',na_filter = False)


def get_i(type):
    int_list = []
    for i in df_i.itertuples(index=True):
        #print(i.code, getattr(i,f'conf_{type}'))
        if getattr(i,f'conf_{type}') not in none_list:
            try:
                documentation = getattr(i,f'doc_{type}') if getattr(i,f'doc_{type}') not in none_list else None
            except AttributeError:
                documentation = None
            int  = construct_fhir_element('CapabilityStatementRestResourceInteraction', dict(
            code = i.code,
            documentation = documentation,
            extension = get_conf(getattr(i,f'conf_{type}')),
            ))
            int_list.append(int.json()),               
    return int_list


def get_sp(r_type):
    sp_list = []
    for i in df_sp.itertuples(index=True):
        if i.base == r_type:
            # TODO need to fix this to reference the package file to reconcile definition to names
            if i.code in sp_specials: #special case temp fix for us-core
                definition = sp_specials[i.code]
            elif i.update == 'Y' or i.exists =='N':
                definition = f'{canon}SearchParameter/{pre.lower()}-{i.base.lower()}-{i.code.split("_")[-1]}'                  
            elif i.code.startswith('_'): #common sp
                definition = f'{fhir_base_url}SearchParameter/{i.code.replace("_","Resource-")}'
            else:  # use base definition
                definition = f'{fhir_base_url}SearchParameter/{i.base}-{i.code}'
            # print(definition)
            
            sp  = construct_fhir_element('CapabilityStatementRestResourceSearchParam', dict(
            name = i.code,
            definition = definition,
            documentation = i.documentation if i.documentation not in none_list else None,               
            type = i.type,
            extension = get_conf(i.base_conf),
            ))
            #print(sp.json())                
            sp_list.append(sp.json())                            
    return sp_list


def get_combo_ext(r_type,combos):
    x_list = []
    for combo in combos:
        # convert to extension
        combo_ext = construct_fhir_element('Extension', dict (
        url = combo_url,
        extension=get_conf(combo[1]),
        ))
        for param in combo[0].split(','):
            req_combo = construct_fhir_element('Extension', dict (
                    url = 'required',
                    valueString = param,   #http://hl7.org/fhir/us/core/SearchParameter/us-core-patient-family
                ))
            combo_ext.extension.append(req_combo)
        x_list.append(combo_ext)
        # print(x_list)
    return x_list
  
    
def get_op(r_type):
    op_list = []
    for i in df_op.itertuples(index=True):
         if i.type == r_type:
            op = construct_fhir_element('CapabilityStatementRestResourceOperation', dict(
            name = i.name, 
            definition = i.definition,
            documentation = i.documentation if i.documentation not in none_list else None,
            extension = get_conf(i.conf),
            ))
            try:                     
                op.extension =  get_addin_ext(op.extension, i.ext)
            except AttributeError:
                print("---- no addin extensions found-----")
            op_list.append(op.json())                           
    return op_list  


rest.resource =  []
for r in df_resources.itertuples(index=True):
    # print(r.type, r.conformance, r.readHistory
    supported_profile = [p.Profile for p in df_profiles.itertuples(index=True) if p.Type == r.type]
    #pprint(supported_profile)                         
    res = construct_fhir_element('CapabilityStatementRestResource', dict(
        type = r.type,
        documentation = set_attr(r.documentation),
        versioning = set_attr(r.versioning),
        readHistory = set_attr(r.readHistory),
        updateCreate = set_attr(r.updateCreate),
        conditionalCreate = set_attr(r.conditionalCreate),
        conditionalRead = set_attr(r.conditionalRead),
        conditionalUpdate = set_attr(r.conditionalUpdate),
        conditionalDelete = set_attr(r.conditionalDelete),
        referencePolicy = set_attr_list(r.referencePolicy),
        searchInclude = set_attr_list(r.include),
        searchInclude__ext = get_prim_ext(r.include_conf),
        searchRevInclude = set_attr_list(r.revinclude),
        searchRevInclude__ext = get_prim_ext(r.revinclude_conf),
        interaction = get_i(r.type),
        searchParam = get_sp(r.type),
        operation = get_op(r.type),
        profile = set_attr(r.profile),
        supportedProfile = supported_profile,
        )
    )
    res.extension = get_conf(r.conformance)
    combos = {(i.combo,i.combo_conf) for i in df_combos.itertuples(index=True) if i.base == r.type}
    res.extension = res.extension + get_combo_ext(r.type,combos) # convert list to  lst of combo extension                              
    rest.resource.append(res)

rest.resource =  sorted(rest.resource,key = lambda x: x.type)  # sort resources                         
cs.rest = [rest]
    
print(cs.json(indent=2))          

### Validate

In [ ]:
 #validate and write to file

print('...validating')
r = validate(cs)
display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

### Create Narrative

- Using Jinja2 Template create xhtml for narrative

#### First: Get spec_internal from package.tgz a json file which includes canonical to local relative page links

Note for this to work you have to have a working build that already contains all the needed artifacts.

In [ ]:
import tarfile
package_path = Path.cwd() / 'tarfiles'/'package.tgz'  #get_si(path)

def get_si(package_path):
    with tarfile.open(package_path, mode='r') as tf:
        #pprint(tf.getnames())
        f = tf.extractfile('package/other/spec.internals')
        r = f.read()
        si = loads(r)
        return si

    
def get_si3(path):
    tf = get(path)
    print(tf)
    return tf

"e.g. https://build.fhir.org/ig/HL7/davinci-deqm/package.tgz" 
try:   
    tf= get_si3(ig_package_tar_path) # get from remote server
except:
   in_path = Path() / ig_package_tar_path /'package.tgz'
   tf = in_path.read_bytes()
   package_path.write_bytes(tf)  # get from package (json) file in local .fhir directory
else:
    package_path.write_bytes(tf.content)    #save in temp file
    
si = get_si(package_path) #unpack from file

path_map = si['paths']
path_map

#### Then Use Jinja2 template to create narrative

In [525]:
in_path = ''
in_file = 'R4capabilitystatement-server.j2'

print(cs.date)

2021-05-26 17:23:23.007182-08:00


In [526]:
def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'],),
    trim_blocks = True,
    lstrip_blocks = True,
    )

env.filters['markdown'] = markdown

template = env.get_template(in_file)

sp_map = {sp.code:sp.type for sp in df_sp.itertuples(index=True)}
pname_map = {p.Profile:p.Name for p in df_profiles.itertuples(index=True)}
#pprint(sp_map)
rendered = template.render(cs=cs, path_map=path_map, pname_map=pname_map, sp_map=sp_map, shall_igs=shall_igs, should_igs=should_igs) #ig_dict=ig_dict )

### Minify the xhtml

In [527]:
def x_minify(xhtml):
    h_min=minify(xhtml, remove_optional_attribute_quotes=False, remove_comments=True)
    x_min = h_min.replace('<br>','<br />')
    x_min = x_min.replace('<hr>','<hr />')
    return x_min

mini = x_minify(rendered)
#print(type(mini))
#display(HTML(rendered))
display(HTML(mini))

'''
#======== write to temp file to debug =======
path = Path.cwd() / 'debug' / 'narrative_pre.xhtml'
path.write_text(rendered, encoding="utf-8")
path = Path.cwd() / 'debug' / 'narrative_mini_pre.xhtml'
path.write_text(mini, encoding="utf-8")
#===================================================
'''
narr = construct_fhir_element('Narrative', dict(
        status = 'generated',
        div = mini,
        ))
cs.text = narr
#print(cs.json(indent=2))

Resource Type,Supported Profiles,Supported Searches,Supported _includes,Supported _revincludes,Supported Operations
AllergyIntolerance,US Core AllergyIntolerance Profile,"clinical-status, patient patient+clinical-status",,Provenance:target,
CarePlan,US Core CarePlan Profile,"category, date, patient, status patient+category, patient+category+date, patient+category+status, patient+category+status+date",,Provenance:target,
CareTeam,US Core CareTeam Profile,"patient, status patient+status",,Provenance:target,
Condition,US Core Condition Profile,"category, clinical-status, patient, onset-date, code patient+category, patient+code, patient+clinical-status, patient+onset-date",,Provenance:target,
Device,US Core Implantable Device Profile,"patient, type patient+type",,Provenance:target,
DiagnosticReport,"US Core DiagnosticReport Profile for Laboratory Results Reporting, US Core DiagnosticReport Profile for Report and Note exchange","status, patient, category, code, date patient+category+date, patient+code+date, patient+category, patient+code, patient+status",,Provenance:target,
DocumentReference,US Core DocumentReference Profile,"_id, status, patient, category, type, date, period patient+category+date, patient+category, patient+status, patient+type, patient+type+period",,Provenance:target,docref
Encounter,US Core Encounter Profile,"_id, class, date, identifier, patient, status, type patient+type, class+patient, patient+status, date+patient",,Provenance:target,
Goal,US Core Goal Profile,"lifecycle-status, patient, target-date patient+lifecycle-status, patient+target-date",,Provenance:target,
Immunization,US Core Immunization Profile,"patient, status, date patient+status, patient+date",,Provenance:target,


### validate again

In [528]:
print('...validating')
r = validate(cs)
d = display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))
           
#======== write to temp file to debug =======
from html.parser import HTMLParser

class HTMLFilter(HTMLParser):
    text = ""
    def handle_data(self, data):
        self.text += data

f = HTMLFilter()
f.feed(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}')
path = Path.cwd() / 'debug' / 'validation.txt'
path.write_text(f.text)
#===================================================


...validating


72670

### Write to folder

In [529]:
# save to file
#save in ig_source folder
ig_source_path = ''

path = Path.cwd() / ig_source_path / 'resources' / f'capabilitystatement-{cs.id.lower()}.json'

#path = Path.cwd() /  'resources' / f'capabilitystatement-{cs.id.lower()}.json' # write locally 


print(f'...........saving to file {path}............')
path.write_text(cs.json(indent=2))

...........saving to file /Users/ehaas/Documents/Python/MyNotebooks/CapStatement/resources/capabilitystatement-us-core-server.json............


231347